In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.optimize import minimize

## Problema principal

Buscamos minimizar las cantidades $$K_1=(4b+c-4a)C_1$$ y $$K_2=(4b+c-4a)C_2$$ provenientes de la función
$$K(\tau)=K_1+K_2\tau^{1/2}-\tfrac{2}{3\sqrt{3}}(\tau-1)^{3/2}.$$
Para esto, establecemos el problema de minimización $(P)$ dado por
$$\min_{(a, b, c, C_1, C_2)\in X}\ K_1^2+K_2^2$$
donde $X\subset \mathbb{R}^5$ es el espacio de restricciones del problema.

## Problema de optimización

A continuación se define la función _objetivo_ del problema de optimización, seguido de las _constraints_ de dicho problema.

*_Observación_*: por comodidad, en el código se consideró $C_1=A^{-1}, C_2=B^{-1}$ y $C_3=C^{1/2}$.

In [2]:
def objective(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return ((4*b+c-4*a)**2)*(1/(A**2)+1/(B**2))
def constraint1(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 14*a*(b-2*a)**4-1.5*(b**3)*(A**2)-1.5*(b+c-a)*a*b*(A**2)-1.5*a*(b**2)*(B**2)-0.75*(b+c-a)*(A**4)-0.125*b*(A**2)*(B**2)
def constraint2(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 6*a*(b-2*a)**2-1.5*b*(A**2)-0.75*a*(B**2)
def constraint3(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 4*(b-2*a)**3-0.25*(b+c-a)*(A**2)-3*(a**2)/((b-2*a)*C)-2*(b-2*a)*(B**2)
def constraint4(x):
    return x[1]-2*x[0]

Se da un punto inicial $x_0$ y se definen las restricciones del problema.

In [3]:
x0=[1, 8, 0.5, 3, 3]

#restricciones
con1={'type':'ineq', 'fun': constraint1}
con2={'type':'ineq', 'fun': constraint2}
con3={'type':'ineq', 'fun': constraint3}
con4={'type':'ineq', 'fun': constraint4}
cons=[con1, con2, con3, con4]

### Primer acercamiento

Fijamos $C_3$ suficientemente grande para resolver el problema. Debido a la libertad de las variables, resolvemos el problema $(P)$ a medida que iteramos sobre las cotas inferiores de $C_1$ y $C_2$. Por la naturaleza del problema original, podemos fijar cotas inferiores para $a, b$ y $c$.

In [4]:
C=10000000 # 10^7
lista=np.linspace(1, 80, 400).tolist()
val_p_min=100
sol_p_min=[10, 10, 10, 10, 10]

for j in lista:
    b1=(1, j)
    b=(0.01, None)
    # Cotas de las variables
    bnds=(b, b, b, b1, b1)
    # No definimos tipo de solver
    sol=minimize(objective, x0, bounds=bnds, constraints=cons)
    # Nos aseguramos de que, en efecto, se satisfagan las restricciones
    # Guardamos la mínima solución
    if sol.fun<val_p_min and constraint1(sol.x)>0 and constraint2(sol.x)>0 and constraint3(sol.x)>0 and constraint4(sol.x)>0:
        val_p_min=sol.fun
        sol_p_min=sol.x
        
print("---------------------------- Solución ----------------------------")
print("val(P)=", val_p_min)
print("Solución del problema: a=%s, b=%s, c=%s, C1=1/%s, C2=1/%s" %tuple(sol_p_min))
print("El valor de K1 es ", (4*sol_p_min[1]+sol_p_min[2]-4*sol_p_min[0])*(1/sol_p_min[3]))
print("El valor de K2 es ", (4*sol_p_min[1]+sol_p_min[2]-4*sol_p_min[0])*(1/sol_p_min[4]))

---------------------------- Solución ----------------------------
val(P)= 71.22234490163238
Solución del problema: a=10.392314734393258, b=90.68876636523513, c=0.01, C1=1/43.30119620959022, C2=1/79.80200501253132
El valor de K1 es  7.417712087414111
El valor de K2 es  4.024908979078033


In [5]:
print(objective(sol_p_min))
print(constraint1(sol_p_min))
print(constraint2(sol_p_min))
print(constraint3(sol_p_min))

71.22234490163238
0.0004653334617614746
1.4260876923799515e-09
438378.02573545603


### Comportamiento de $val(P)$

Al disminuir la cota inferior de $C_1$ y $C_2$ vemos que $val(P)$ tiene chance de decrecer, debido a que la función objetivo es decreciente respecto a dichos parámetros. En efecto, en la siguiente iteración, fijando como cota inferior $10^3$ podemos apreciar tal comportamiento, y a su vez, obtener la misma conclusión tanto para $K_1$ como para $K_2$.

In [6]:
lista=np.linspace(1, 1000).tolist()
for j in lista:
    b1=(1, j)
    b=(0.01, None)
    # Cotas de las variables
    bnds=(b, b, b, b1, b1)
    # No definimos tipo de solver
    sol=minimize(objective, x0, bounds=bnds, constraints=cons)
    # Nos aseguramos de que, en efecto, se satisfagan las restricciones
    if constraint1(sol.x)>0 and constraint2(sol.x)>0 and constraint3(sol.x)>0 and constraint4(sol.x)>0:
        print("---------------------------- Nueva iteración ----------------------------")
        print("val(P)=", sol.fun)
        print("El valor de K1 es ", (4*sol.x[1]+sol.x[2]-4*sol.x[0])*(1/sol.x[3]))
        print("El valor de K2 es ", (4*sol.x[1]+sol.x[2]-4*sol.x[0])*(1/sol.x[4]))

---------------------------- Nueva iteración ----------------------------
val(P)= 71.23647262081964
El valor de K1 es  7.418278281311789
El valor de K2 es  4.025620444333699
---------------------------- Nueva iteración ----------------------------
val(P)= 71.22705334648721
El valor de K1 es  7.417899059220794
El valor de K2 es  4.025149300795993
---------------------------- Nueva iteración ----------------------------
val(P)= 71.22217270434035
El valor de K1 es  7.417646359605643
El valor de K2 es  4.025008719017826
---------------------------- Nueva iteración ----------------------------
val(P)= 71.22118941950248
El valor de K1 es  7.417005204715243
El valor de K2 es  4.026067959278567
---------------------------- Nueva iteración ----------------------------
val(P)= 71.22052885531701
El valor de K1 es  7.4168758366939675
El valor de K2 es  4.026224245913556
---------------------------- Nueva iteración ----------------------------
val(P)= 71.2200570068944
El valor de K1 es  7.416746576

### Segundo acercamiento

Se conjetura que $val(P)>71.22$ (en base a una cota inferior para $C_1, C_2$ de $10^{-3}$). Al disminuir la cota inferior de las variables $C_1$ y $C_2$, las mejoras obtenidas en la cota inferior del parámetro $\eta$ son del orden de $10^{-3}$, lo cual consideramos no significativo. Considerar además que el solver puede no dar buenos resultados con números de orden menor.

Establecido este rango, vemos que es más significativo disminuir $K_2$ sin que aumente (demasiado) $K_1$, por lo cual consideramos, esencialmente, un nuevo problema de optimización $(S)$ (que depende de $C_1$) donde la función objetivo es $K_2$, con las mismas restricciones del problema $(P)$ pero con una nueva restricción que capture el rango de valores mínimos del problema inicial. En otras palabras $$(S)\ \min_{(a, b, c, C_2)\in Y}\ K_2$$
donde $Y\subset \mathbb{R}^4$ es el espacio de restricciones del problema.

In [7]:
def objective(x):
    a=x[0]
    b=x[1]
    c=x[2]
    B=x[3]
    return (4*b+c-4*a)/B
def constraint1(x):
    a=x[0]
    b=x[1]
    c=x[2]
    B=x[3]
    return 14*a*(b-2*a)**4-1.5*(b**3)*(A**2)-1.5*(b+c-a)*a*b*(A**2)-1.5*a*(b**2)*(B**2)-0.75*(b+c-a)*(A**4)-0.125*b*(A**2)*(B**2)
def constraint2(x):
    a=x[0]
    b=x[1]
    c=x[2]
    B=x[3]
    return 6*a*(b-2*a)**2-1.5*b*(A**2)-0.75*a*(B**2)
def constraint3(x):
    a=x[0]
    b=x[1]
    c=x[2]
    B=x[3]
    return 4*(b-2*a)**3-0.25*(b+c-a)*(A**2)-3*(a**2)/((b-2*a)*C)-2*(b-2*a)*(B**2)
def constraint4(x):
    return x[1]-2*x[0]
def constraint5(x):
    a=x[0]
    b=x[1]
    c=x[2]
    B=x[3]
    return 7.6*A-(4*b+c-4*a)

x0=[1, 8, 0.5, 3]

# Añadimos la nueva restricción
con1={'type':'ineq', 'fun': constraint1}
con2={'type':'ineq', 'fun': constraint2}
con3={'type':'ineq', 'fun': constraint3}
con4={'type':'ineq', 'fun': constraint4}
con5={'type':'ineq', 'fun': constraint5}
cons=[con1, con2, con3, con4, con5]

C=10000000 # 10^7
lista=np.linspace(1, 80, 400).tolist()
list_A=np.linspace(1.5, 2, 6).tolist()
val_p_min=100
sol_p_min=[10, 10, 10, 10]

for A in list_A:
    for j in lista:
        b1=(1, j)
        b=(0.01, None)
        # Cotas de las variables
        bnds=(b, b, b, b1)
        # No definimos tipo de solver
        sol=minimize(objective, x0, bounds=bnds, constraints=cons)
        # Nos aseguramos de que, en efecto, se satisfagan las restricciones
        # Guardamos la mínima solución
        if sol.fun<val_p_min and constraint1(sol.x)>0 and constraint2(sol.x)>0 and constraint3(sol.x)>0 and constraint4(sol.x)>0 and constraint5(sol.x)>0:
            val_p_min=sol.fun
            sol_p_min=sol.x
    print("---------------------------- Solución ----------------------------")
    print("val(P)=", val_p_min)
    print("C1=1/%s", A)
    print("Solución del problema: a=%s, b=%s, c=%s, C2=1/%s" %tuple(sol_p_min))
    print("El valor de K1 es ", (4*sol_p_min[1]+sol_p_min[2]-4*sol_p_min[0])*(1/A))
    print("El valor de K2 es ", (4*sol_p_min[1]+sol_p_min[2]-4*sol_p_min[0])*(1/sol_p_min[3]))

---------------------------- Solución ----------------------------
val(P)= 4.009371963386333
C1=1/%s 1.5
Solución del problema: a=0.35967547739936895, b=3.145648459453747, c=0.010000000000000174, C2=1/2.7819548872180184
El valor de K1 es  7.435927952145008
El valor de K2 es  4.009371963386333
---------------------------- Solución ----------------------------
val(P)= 3.805469662879118
C1=1/%s 1.6
Solución del problema: a=0.3819801470829627, b=3.402873337891685, c=0.010000000000002328, C2=1/3.1779448621553885
El valor de K1 es  7.558482977021806
El valor de K2 es  3.8054696628791174
---------------------------- Solución ----------------------------
val(P)= 3.805469662879118
C1=1/%s 1.7
Solución del problema: a=0.3819801470829627, b=3.402873337891685, c=0.010000000000002328, C2=1/3.1779448621553885
El valor de K1 es  7.113866331314641
El valor de K2 es  3.8054696628791174
---------------------------- Solución ----------------------------
val(P)= 3.805469662879118
C1=1/%s 1.8
Solución del 

Iteramos entre $A=1.5$ y $A=2$ con un paso de $0.1$, pues a partir de ahí no se obtiene una mejora en $val(S)$.

Con lo siguiente vemos que una elección aproximada de $argmin(S)$ satisface las restricciones iniciales.

In [8]:
sol=[0.4768, 4.2744, 0.01,  4.0622]
print("argmin(P) aproximado:", sol)
print("El valor de K1 es ", (4*sol[1]+sol[2]-4*sol[0])*(1/A))
print("El valor de K2 es ", (4*sol[1]+sol[2]-4*sol[0])*(1/sol[3]))
print("val(P) app-val(P):", objective(sol)-objective(sol_p_min))
print("C1(argmin(P)) aproximado:", constraint1(sol))
print("C2(argmin(P)) aproximado:", constraint2(sol))
print("C3(argmin(P)) aproximado:", constraint3(sol))
print("C4(argmin(P)) aproximado:", constraint4(sol))

argmin(P) aproximado: [0.4768, 4.2744, 0.01, 4.0622]
El valor de K1 es  7.600200000000001
El valor de K2 es  3.741913248978387
val(P) app-val(P): 0.0001110279447189555
C1(argmin(P)) aproximado: 0.057715418761567605
C2(argmin(P)) aproximado: 0.0007550633280040131
C3(argmin(P)) aproximado: 33.07955707136637
C4(argmin(P)) aproximado: 3.3208


Asimismo, vemos que con dicha elección no estamos muy lejos de $val(P)$ original.

In [9]:
sol_app=[0.4768, 4.2744, 0.01, 2, 4.0622]
def objP(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return ((4*b+c-4*a)**2)*(1/(A**2)+1/(B**2))
print(objP(sol_app))

71.76495480288


## Conclusiones

Para $C_1, C_2\geq 1/80$, seleccionamos los parámetros aproximados a cuatro decimales
$$a=0.4768,\ b=4.2744,\ c=0.01,\ C_1=1/2,\ C_2=1/4.0622.$$
Con dicha elección la función objetivo evaluada en estos puntos es mayor a $val(P)$ y difiere en aproximadamente $1.1\cdot 10^{-4}$. A partir de ello, en nuestro problema principal (en el cual interfieren otros parámetros que no se mencionan acá) $\eta\geq 23.25$.